# CNN scraping
This notebook aims to build a scaper of the news in https://www.cnnbrasil.com.br/

In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from bs4 import BeautifulSoup
from time import sleep
from selenium.webdriver.common.by import By
import pandas as pd
from datetime import datetime, timedelta

from src import scrapers
# Path to geckodriver executable
geckodriver_path = '/snap/bin/firefox.geckodriver'


s = Service(executable_path=geckodriver_path)
# Create a Firefox webdriver instance
# opens a window
driver = webdriver.Firefox(service=s)

In [2]:
scraper = scrapers.CNNNewsScraper()
data = scraper.get_scraped_data(driver)
data

Scroll 1, height=5780
Scroll 2, height=8661
Scroll 3, height=11481
Scroll 4, height=14301
Scroll 5, height=17121
Error clicking
Scroll 6, height=19941
Error clicking
Scroll 7, height=19941
Error clicking
Scroll 8, height=19941
Error clicking
Scroll 9, height=19941
Error clicking
Scroll 10, height=19941
Scroll 1, height=6585
Scroll 2, height=9588
Scroll 3, height=12408
Scroll 4, height=15228
Scroll 5, height=18048
Error clicking
Scroll 6, height=20868
Error clicking
Scroll 7, height=20868
Error clicking
Scroll 8, height=20868
Error clicking
Scroll 9, height=20868
Error clicking
Scroll 10, height=20868
Error clicking
Scroll 1, height=8403
Error clicking
Scroll 2, height=9601
Error clicking
Scroll 3, height=9601
Error clicking
Scroll 4, height=9601
Error clicking
Scroll 5, height=9601
Error clicking
Scroll 6, height=9601
Error clicking
Scroll 7, height=9601
Error clicking
Scroll 8, height=9601
Error clicking
Scroll 9, height=9601
Error clicking
Scroll 10, height=9601
Scroll 1, height=5532

,Title,Time,Theme
0,Protestos marcam início de reunião do governo...,11/07/2024 às 22:16,politica
1,“Abin paralela” monitorou familiares de Jean ...,11/07/2024 às 22:09,politica
2,Análise: A anistia a partidos e o teatro da p...,11/07/2024 às 22:03,politica
3,PGR rejeita pedido para incluir “gabinete da ...,11/07/2024 às 21:35,politica
4,"Após pedido de Xuxa, Haddad e Janja divulgam ...",11/07/2024 às 21:20,politica
...,...,...,...
55,\n\t\t\t\t\t\tMarina Ruy Barbosa parabeniza Ju...,\n\t\t\t\t\t\t10/07/2024 às 21:12\t\t\t\t\t,pop
56,"\n\t\t\t\t\t\tDe Iza a Deborah Secco, relembre...",\n\t\t\t\t\t\t10/07/2024 às 20:46\t\t\t\t\t,pop
57,\n\t\t\t\t\t\tFamosos reagem à separação de Iz...,\n\t\t\t\t\t\t10/07/2024 às 20:40\t\t\t\t\t,pop
58,\n\t\t\t\t\t\tCasamento às Cegas: Marília diz ...,\n\t\t\t\t\t\t10/07/2024 às 20:28\t\t\t\t\t,pop


In [2]:
driver.get('https://www.cnnbrasil.com.br/')

In [3]:
def scroll_page(driver, n_scrolls):
    current_height = driver.execute_script("return document.body.scrollHeight")

    for i in range(n_scrolls):
        # scroll to the end of the page
        driver.execute_script(f"window.scrollTo(0,document.body.scrollHeight)")
        sleep(1)
        
        # handle the 'Aceitar Cokkies' popup
        # cookies_button = driver.find_elements(By.CSS_SELECTOR, value='#onetrust-reject-all-handler')
        # if cookies_button:
        #     cookies_button[0].click()
        #     sleep(1)
        
        try:
            driver.find_element(By.CSS_SELECTOR, value='.block-list-get-more-btn').click()
        except Exception:
            print('Error clicking')
        
        
        
        # repeat the scroll
        driver.execute_script(f"window.scrollTo(0,document.body.scrollHeight - 1000)")
        current_height = driver.execute_script("return document.body.scrollHeight")
            
        print(f'Scroll {i + 1}, height={current_height}')
        
        # time for the page to load
        sleep(4)

In [4]:
def scrape_news_from_page(driver, theme):
    titles = []
    times = []
    themes = []

    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'lxml')
    content_block = soup.find('div', class_ = 'col__l--9 col--12')
    news_list = content_block.find_all('li', class_ = 'home__list__item')
    
    for news in news_list:
        title = news.find('h3', class_ = 'news-item-header__title')
        titles.append(title.text)

        time = news.find('span', 'home__title__date')
        times.append(None if time is None else time.text)
        
        themes.append(theme)
        
    data = pd.DataFrame({
        'Title': titles,
        'Time': times,
        'Theme': themes
    })

    return data

In [5]:
themes = ['politica', 'economia', 'esportes', 'pop']

In [6]:
data_dfs = []

for theme in themes:
    driver.get(f'https://www.cnnbrasil.com.br/{theme}')
    sleep(5)
    
    scroll_page(driver, 10)
    
    data = scrape_news_from_page(driver, theme)
    
    data_dfs.append(data)

Scroll 1, height=8049
Scroll 2, height=11581
Scroll 3, height=15161
Scroll 4, height=18669
Scroll 5, height=22081
Error clicking
Scroll 6, height=25661
Error clicking
Scroll 7, height=25661
Error clicking
Scroll 8, height=25661
Error clicking
Scroll 9, height=25661
Error clicking
Scroll 10, height=25661
Scroll 1, height=9610
Scroll 2, height=13155
Scroll 3, height=16591
Scroll 4, height=20099
Scroll 5, height=23679
Error clicking
Scroll 6, height=27211
Error clicking
Scroll 7, height=27211
Error clicking
Scroll 8, height=27211
Error clicking
Scroll 9, height=27139
Error clicking
Scroll 10, height=27211
Error clicking
Scroll 1, height=10197
Scroll 2, height=14933
Error clicking
Scroll 3, height=19076
Error clicking
Scroll 4, height=19076
Error clicking
Scroll 5, height=19076
Error clicking
Scroll 6, height=19076
Error clicking
Scroll 7, height=19076
Error clicking
Scroll 8, height=19076
Error clicking
Scroll 9, height=19076
Error clicking
Scroll 10, height=19076
Scroll 1, height=11308
S

In [7]:
data = pd.concat(data_dfs, axis='rows')

In [8]:
def convert_to_datetime(time_str):
    # Remove the 'às' and strip any extra whitespace
    cleaned_str = time_str.strip().replace(' às ', ' ')
    # Convert to datetime using the appropriate format
    return datetime.strptime(cleaned_str, '%d/%m/%Y %H:%M')

# Apply the function to the series
data['Time'] = data['Time'].apply(convert_to_datetime)

data['Title'] = data['Title'].str.strip()

data

,Title,Time,Theme
0,PGR rejeita pedido para incluir “gabinete da o...,2024-07-11 21:35:00,politica
1,"Após pedido de Xuxa, Haddad e Janja divulgam v...",2024-07-11 21:20:00,politica
2,Mulher de Eduardo Bolsonaro o compara a cantor...,2024-07-11 21:08:00,politica
3,Abin paralela: Servidores de carreira defendem...,2024-07-11 21:02:00,politica
4,Mais de 30 entidades assinam nota contra PEC d...,2024-07-11 20:56:00,politica
...,...,...,...
105,Dani Calabresa revela que aceitou R$ 300 para ...,2024-07-09 17:02:00,pop
106,"Morre Adrián Olivares, ex-integrante do grupo ...",2024-07-09 16:46:00,pop
107,"“Os Bandidos do Tempo”, remake do filme de 198...",2024-07-09 16:29:00,pop
108,“Mania de Você”: saiba como será a personagem ...,2024-07-09 16:18:00,pop


In [10]:
data.Theme.value_counts()

pop         110
politica     60
economia     60
esportes     20
Name: Theme, dtype: int64